# Load saved data

In [1]:
import h5py
from sklearn.model_selection import train_test_split

def loadNew():
    f = h5py.File("yelpdataset12345.h5")
    x = f['x'].value
    y = f['y'].value
    f.close()
    x_train , x_testInitial, y_train, y_testInitial = train_test_split(x,y,test_size=0.3,random_state=100)
    
    x_val , x_test, y_val, y_test = train_test_split(x_testInitial,y_testInitial,test_size=0.5,random_state=100)
    return x_train, x_val, y_train, y_val

In [2]:
import numpy as np
#from getdata import load
from keras.models import Sequential
from getdata import load
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import hamming_loss
from keras import backend as K
K.set_image_dim_ordering('th')

x_train, x_val, y_train, y_val = loadNew()

x_train = x_train.astype('float32')
x_val  = x_val.astype('float32')

x_train /= 255
x_val /= 255



Using TensorFlow backend.


In [6]:
x_val

array([[[[ 0.10588235,  0.12156863,  0.07843138, ...,  0.08627451,
           0.11764706,  0.11764706],
         [ 0.08235294,  0.09019608,  0.10588235, ...,  0.14509805,
           0.08235294,  0.09411765],
         [ 0.06666667,  0.07843138,  0.09019608, ...,  0.08627451,
           0.10980392,  0.14509805],
         ..., 
         [ 0.15686275,  0.12941177,  0.11372549, ...,  0.14901961,
           0.11764706,  0.11764706],
         [ 0.09411765,  0.08235294,  0.09803922, ...,  0.12156863,
           0.16862746,  0.15294118],
         [ 0.07058824,  0.07058824,  0.10980392, ...,  0.13725491,
           0.10588235,  0.09019608]],

        [[ 0.03921569,  0.04705882,  0.02745098, ...,  0.06666667,
           0.05882353,  0.01960784],
         [ 0.03137255,  0.02352941,  0.05098039, ...,  0.1254902 ,
           0.10196079,  0.07843138],
         [ 0.03921569,  0.02352941,  0.02352941, ...,  0.05098039,
           0.05490196,  0.05098039],
         ..., 
         [ 0.05882353,  0.074509

# Prepare Keras sequential model

In [7]:
model = Sequential()
model.add(Convolution2D(32, kernel_size=(3, 3),padding='same',input_shape=(3 , 100, 100)))
model.add(Activation('relu'))
model.add(Convolution2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Convolution2D(64,(3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(9))
model.add(Activation('sigmoid'))




# model.load_weights("weights.16-0.86800.hdf5")


/Users/ajinkya.parkar@ibm.com/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3))`
  # This is added back by InteractiveShellApp.init_path()


In [8]:
# let's train the model using SGD + momentum (how original).
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])

# Output labels

In [9]:
y_val

array([[0, 0, 1, ..., 1, 0, 0],
       [0, 1, 1, ..., 1, 1, 1],
       [1, 1, 0, ..., 1, 0, 1],
       ..., 
       [0, 1, 1, ..., 1, 1, 0],
       [0, 1, 0, ..., 1, 0, 0],
       [0, 1, 1, ..., 1, 1, 1]])

In [10]:

check = ModelCheckpoint("weights.{epoch:02d}-{val_acc:.5f}.hdf5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=True, mode='auto')


# Train Model

In [11]:
model.fit(x_train, y_train, batch_size=100, nb_epoch=20,callbacks=[check],validation_data=(x_val,y_val))


/Users/ajinkya.parkar@ibm.com/anaconda/lib/python3.6/site-packages/keras/models.py:848: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 164389 samples, validate on 35226 samples
Epoch 1/20
164389/164389 [==============================] - 18209s - loss: 0.5973 - acc: 0.6725 - val_loss: 0.5722 - val_acc: 0.6920
Epoch 2/20
164389/164389 [==============================] - 17867s - loss: 0.5713 - acc: 0.6929 - val_loss: 0.5606 - val_acc: 0.7008
Epoch 3/20
164389/164389 [==============================] - 18527s - loss: 0.5611 - acc: 0.7005 - val_loss: 0.5522 - val_acc: 0.7062
Epoch 4/20
164389/164389 [==============================] - 18658s - loss: 0.5543 - acc: 0.7054 - val_loss: 0.5551 - val_acc: 0.7041
Epoch 5/20
164389/164389 [==============================] - 18363s - loss: 0.5487 - acc: 0.7093 - val_loss: 0.5437 - val_acc: 0.7129
Epoch 6/20
164389/164389 [==============================] - 18200s - loss: 0.5438 - acc: 0.7129 - val_loss: 0.5468 - val_acc: 0.7111
Epoch 7/20
164389/164389 [==============================] - 18273s - loss: 0.5391 - acc: 0.7167 - val_loss: 0.5379 - val_acc: 0.7171
Epoch 8/20
164389/

In [48]:
x_train[0][0][0]

array([ 0.05882353,  0.07450981,  0.05882353,  0.05490196,  0.06666667,
        0.08627451,  0.07843138,  0.07843138,  0.08235294,  0.09411765,
        0.09803922,  0.10588235,  0.11372549,  0.12156863,  0.1254902 ,
        0.12156863,  0.12941177,  0.14117648,  0.14117648,  0.13725491,
        0.14117648,  0.15686275,  0.16078432,  0.15294118,  0.15686275,
        0.14509805,  0.14509805,  0.14117648,  0.14117648,  0.14901961,
        0.07450981,  0.14509805,  0.14117648,  0.15686275,  0.14509805,
        0.15294118,  0.15686275,  0.16078432,  0.16862746,  0.16862746,
        0.17254902,  0.17647059,  0.17254902,  0.16470589,  0.15686275,
        0.16470589,  0.17647059,  0.17647059,  0.17647059,  0.17647059,
        0.16470589,  0.16470589,  0.16862746,  0.16862746,  0.18039216,
        0.18039216,  0.18039216,  0.18039216,  0.18039216,  0.18039216,
        0.18039216,  0.18039216,  0.17647059,  0.17254902,  0.17254902,
        0.17254902,  0.17254902,  0.16862746,  0.16862746,  0.17

In [49]:
y_train

array([[0, 0, 1, 1, 0],
       [0, 0, 1, 0, 0],
       [0, 1, 0, 0, 0],
       ..., 
       [0, 0, 0, 0, 1],
       [0, 1, 0, 0, 1],
       [0, 0, 0, 1, 0]])

In [52]:
x_train[0][0][0]

array([ 0.52941179,  0.5411765 ,  0.50196081,  0.49411765,  0.40392157,
        0.43529412,  0.42745098,  0.41176471,  0.42745098,  0.43921569,
        0.49411765,  0.44705883,  0.41960785,  0.41960785,  0.41960785,
        0.44705883,  0.4509804 ,  0.41568628,  0.42745098,  0.44705883,
        0.45490196,  0.40000001,  0.32549021,  0.23921569,  0.4509804 ,
        0.43137255,  0.45490196,  0.45490196,  0.4627451 ,  0.4627451 ,
        0.47058824,  0.45490196,  0.47450981,  0.4627451 ,  0.4627451 ,
        0.4509804 ,  0.44313726,  0.4509804 ,  0.45882353,  0.45882353,
        0.44705883,  0.47058824,  0.47843137,  0.4509804 ,  0.47058824,
        0.49803922,  0.51372552,  0.4627451 ,  0.53725493,  0.54509807,
        0.45490196,  0.47058824,  0.4509804 ,  0.46666667,  0.49803922,
        0.49019608,  0.51372552,  0.51372552,  0.48627451,  0.49803922,
        0.50196081,  0.51764709,  0.54901963,  0.5529412 ,  0.50980395,
        0.50980395,  0.49019608,  0.52941179,  0.49019608,  0.52

In [54]:
y_train[0]

array([0, 1, 1, 1, 0, 1, 1, 0, 0])

# Model Summary

In [60]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_24 (Conv2D)           (None, 32, 100, 100)      896       
_________________________________________________________________
activation_33 (Activation)   (None, 32, 100, 100)      0         
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 64, 98, 98)        18496     
_________________________________________________________________
activation_34 (Activation)   (None, 64, 98, 98)        0         
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 64, 49, 49)        0         
_________________________________________________________________
dropout_16 (Dropout)         (None, 64, 49, 49)        0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 64, 49, 49)        36928     
__________